<a href="https://colab.research.google.com/github/dorvalneto/ProjetoImersaoIA/blob/main/Projeto_Imersao_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importa as bibliotecas

In [42]:
import pathlib
import textwrap

import google.generativeai as genai

import requests

from IPython.display import display
from IPython.display import Markdown

import json

from collections import defaultdict

### Gera funcionalidade de mostrar com Markdown

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Configura a chave de Acesso da API do Google Gemini

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY= userdata.get('SECRET_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Configura parâmetros da API do Google Gemini

In [62]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}

safety_settings = {
    "harassment": "block_none",
    "hate": "block_none",
    "sexual": "block_none",
    "dangerous": "block_none"
}

### Configura a chave de Acesso da API do Portal da Transparência

In [32]:
from google.colab import userdata

TRANSP_API_KEY= userdata.get('TRANSP_KEY')

headers = {
    'Content-Type': 'application/json;charset=ISO-8859-1',
    'chave-api-dados':TRANSP_API_KEY
}

### Função para fazer o Merge de JSON

In [53]:
def merge_json(d1, d2):
    dd = defaultdict(list)
    if (d1 == '{}'):
      return dict(d2)
    else:
      for d in (d1, d2):
          for key, value in d.items():
              if isinstance(value, list):
                  dd[key].extend(value)
              else:
                  dd[key].append(value)
      return dict(dd)

### Função para buscar nota fiscal por chave no Portal da Transparência

In [59]:
def buscar_nota_por_chave(chaveNotaFiscal):
  url = f"http://api.portaldatransparencia.gov.br/api-de-dados/notas-fiscais-por-chave?chaveUnicaNotaFiscal={chaveNotaFiscal}"
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
      data = response.json()
      return(data)
  else:
      print(f"Error(notas-fiscais-por-chave): {response.status_code}{response.text}")

### Pesquisa no Portal da Transparência

In [66]:
print("\n*** Pesquisa no Portal da Transparência ***")
produto = input("\nQual Produto você gostaria de consultar?")
print("\n*** A pesquisa pode demorar, portanto quanto mais páginas, maior a espera... ***")
quantidade = input("\nQuantas páginas de Notas Fiscais você quer consultar?")

# Inicializa a variável todasNotas
todasNotas = '['

# Inicia o processamento
if(quantidade.isnumeric):
  quantidade = int(quantidade)

  for pagina in range(1, quantidade+1):

    url = f"http://api.portaldatransparencia.gov.br/api-de-dados/notas-fiscais?nomeProduto={produto}&pagina={pagina}"
    print("\nConsultando...")
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
            data = response.json()
            # Processa a resposta em JSON
            # Busca os detalhes de cada Nota Fiscal encontrada
            for item in data:
                notaCompleta = buscar_nota_por_chave(item['chaveNotaFiscal'])
                #todasNotas = merge_json(todasNotas, notaCompleta)
                todasNotas += str(notaCompleta) + ','
            else:
              todasNotas += ']'
              print("\nConsulta concluída!")
    else:
            print(f"Error(notas-fiscais): {response.status_code}{response.text}")
  else:
    print("")
else:
  print("A quantidade informada não é numérica.")


*** Pesquisa no Portal da Transparência ***

Qual Produto você gostaria de consultar?Arroz

*** A pesquisa pode demorar, portanto quanto mais páginas, maior a espera... ***

Quantas páginas de Notas Fiscais você quer consultar?1

Consultando...

Consulta concluída!



### Faz a pesquisa com o Google Gemini sobre os itens recuperados

In [70]:
print("\n*** Pesquisa no GOOGLE GEMINI ***")

print("\n*** Nesse CHAT você vai perguntar sobre as notas pesquisadas anteriormente ***")
print("\n -> Para finalizar digite a palavra - fim - ")

model = genai.GenerativeModel("gemini-1.0-pro",
                              safety_settings=safety_settings,
                              generation_config=generation_config)

chat = model.start_chat(history=[])

prompt = input("Esperando o prompt: ")

while prompt != "fim":
  promptCompleto = f"Dado o seguinte JSON:{todasNotas}, Responda o seguinte questionamento: {prompt}"
  response = chat.send_message(promptCompleto)
  print(f"Resposta: ", response.text, "\n")
  prompt = input("Esperando o prompt: ")
else:
  #Imprimindo o histórico
  print("\n")
  to_markdown("**IMPRIMINDO O HISTÓRICO**")
  print("------")
  for message in chat.history:
    display(to_markdown(f"**{message.role}:** {message.parts[0].text}"))
    print("------")


*** Pesquisa no GOOGLE GEMINI ***

*** Nesse CHAT você vai perguntar sobre as notas pesquisadas anteriormente ***

 -> Para finalizar digite a palavra - fim - 
Esperando o prompt: qual e o valor unitario desses produtos?
Resposta:  - Azeite De Dende 100ml Cepera UN: 6,65
- Molho De Tomate 340g Tradicional Quero UN: 1,79
- Creme De Leite 200g Leve Tp Terra Viva UN: 2,27
- Canela Em Po 30gr Tubo Kamila UN: 2,55
- Cenoura Kg Feira Quadri KG: 6,65
- Ervilha 300g Pct / Congelado Lar* UN: 7,60
- Feijão Kg Feira Quadri KG: 4,52
- Leite 1 Lt Zero Lactose Terra Viva UN: 3,12
- Chocolate Granulado 50g Kamila UN: 1,89
- Peito De Frango Kg C/ Osso Quadri. UN: 12,34
- Vagem Kg Feira Quadri KG: 13,33
- Tomate Kg Longa Vida Feira Quadri KG: 1,93
- Maionese 500g Pet/ Light Hellmanns UN: 8,54
- Iogurte Copo 165g Natural Integral Frimesa UN: 2,27
- Alface Un Crespa Feira Quadri UN: 3,79
- Farinha De Trigo 5kg Bonina UN: 16,61
- Farinha De Trigo 1kg Integral Anaconda UN: 3,79
- Farinha De Milho 500g Mim

> **user:** Dado o seguinte JSON:[{'notaFiscalDTO': {'id': 6028, 'codigoOrgaoSuperiorDestinatario': '-1', 'orgaoSuperiorDestinatario': 'Sem informação', 'codigoOrgaoDestinatario': '-1', 'orgaoDestinatario': 'Sem informação', 'nomeFornecedor': 'QUADRI SUPERMERCADOS LTDA', 'cnpjFornecedor': '22.683.934/0003-70', 'municipioFornecedor': 'REALEZA', 'chaveNotaFiscal': '41220222683934000370550010000154841114629025', 'valorNotaFiscal': '190,73', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '04/02/2022 10:31:37', 'dataEmissao': '04/02/2022', 'numero': 15484, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '21', 'descricaoProdutoServico': 'Azeite De Dende 100ml Cepera UN', 'codigoNcmSh': '15119000', 'ncmSh': 'Outros óleos de dende', 'cfop': '5929', 'quantidade': '3,00', 'unidade': 'UN', 'valorUnitario': '6,65', 'valor': '19,95'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Molho De Tomate 340g Tradicional Quero UN', 'codigoNcmSh': '21032010', 'ncmSh': 'Ketchup e outros molhos de tomate, preparado, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '1,79', 'valor': '1,79'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Creme De Leite 200g Leve Tp Terra Viva UN', 'codigoNcmSh': '4015021', 'ncmSh': 'Creme de leite UHT (Ultra High Temperature), com um teor, em peso, de matérias gordas, superior a 10 %, não concentrados nem adicionados de açúcar ou de outros edulcorantes', 'cfop': '5929', 'quantidade': '4,00', 'unidade': 'UN', 'valorUnitario': '2,27', 'valor': '9,08'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Canela Em Po 30gr Tubo Kamila UN', 'codigoNcmSh': '9062000', 'ncmSh': 'Canela e flores de caneleira, trituradas ou em pó', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '2,55', 'valor': '2,55'}, {'numeroProduto': '20', 'descricaoProdutoServico': 'Cenoura Kg Feira Quadri KG', 'codigoNcmSh': '7061000', 'ncmSh': 'Cenouras e nabos, frescos ou refrigerados', 'cfop': '5929', 'quantidade': '0,68', 'unidade': 'KG', 'valorUnitario': '6,65', 'valor': '4,52'}, {'numeroProduto': '22', 'descricaoProdutoServico': 'Ervilha 300g Pct / Congelado Lar* UN', 'codigoNcmSh': '7102100', 'ncmSh': 'Ervilhas, não cozidas ou cozidas em água ou vapor, congeladas', 'cfop': '5929', 'quantidade': '2,00', 'unidade': 'UN', 'valorUnitario': '7,60', 'valor': '15,20'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Mel 550g Pinduca UN', 'codigoNcmSh': '4090000', 'ncmSh': 'Mel natural', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '21,84', 'valor': '21,84'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Leite 1 Lt Zero Lactose Terra Viva UN', 'codigoNcmSh': '4012010', 'ncmSh': 'Leite UHT (Ultra High Temperature), com um teor, em peso, de matérias gordas, superior a 1 %, mas não superior a 6 %, não concentrados nem adicionados de açúcar ou de outros edulcorantes', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '3,12', 'valor': '3,12'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Chocolate Granulado 50g Kamila UN', 'codigoNcmSh': '18063210', 'ncmSh': 'Chocolate não recheado, em tabletes, barras e paus', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '1,89', 'valor': '1,89'}, {'numeroProduto': '18', 'descricaoProdutoServico': 'Peito De Frango Kg C/ Osso Quadri. UN', 'codigoNcmSh': '2071400', 'ncmSh': 'Pedaços e miudezas, comestíveis de galos/galinhas, congelados', 'cfop': '5929', 'quantidade': '2,15', 'unidade': 'UN', 'valorUnitario': '12,34', 'valor': '26,53'}, {'numeroProduto': '19', 'descricaoProdutoServico': 'Vagem Kg Feira Quadri KG', 'codigoNcmSh': '7089000', 'ncmSh': 'Outros legumes de vagem, frescos ou refrigerados', 'cfop': '5929', 'quantidade': '0,23', 'unidade': 'KG', 'valorUnitario': '13,33', 'valor': '3,07'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Farinha De Trigo 1kg Integral Anaconda UN', 'codigoNcmSh': '11010010', 'ncmSh': 'Farinha de trigo', 'cfop': '5929', 'quantidade': '4,00', 'unidade': 'UN', 'valorUnitario': '5,59', 'valor': '22,36'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Arroz 1kg Branco Itasa UN', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '3,79', 'valor': '3,79'}, {'numeroProduto': '23', 'descricaoProdutoServico': 'Tomate Kg Longa Vida Feira Quadri KG', 'codigoNcmSh': '7020000', 'ncmSh': 'Tomates, frescos ou refrigerados', 'cfop': '5929', 'quantidade': '0,68', 'unidade': 'KG', 'valorUnitario': '2,84', 'valor': '1,93'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Farinha De Mandioca 1kg Branca Pinduca UN', 'codigoNcmSh': '11062000', 'ncmSh': 'Farinhas, sêmolas e pós, de sagu ou das raízes ou tubérculos, da posição 07.14', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '4,74', 'valor': '4,74'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Farinha De Trigo 5kg Bonina UN', 'codigoNcmSh': '11010010', 'ncmSh': 'Farinha de trigo', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '16,61', 'valor': '16,61'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Farinha De Milho 500g Mimoso Zaeli UN', 'codigoNcmSh': '11022000', 'ncmSh': 'Farinha de milho', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '2,74', 'valor': '2,74'}, {'numeroProduto': '16', 'descricaoProdutoServico': 'Iogurte Copo 165g Natural Integral Frimesa UN', 'codigoNcmSh': '4031000', 'ncmSh': 'Iogurte', 'cfop': '5929', 'quantidade': '3,00', 'unidade': 'UN', 'valorUnitario': '2,27', 'valor': '6,81'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Chimichurri 40g Kamila UN', 'codigoNcmSh': '21039029', 'ncmSh': 'Outros condimentos e temperos, compostos', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '3,31', 'valor': '3,31'}, {'numeroProduto': '17', 'descricaoProdutoServico': 'Alface Un Crespa Feira Quadri UN', 'codigoNcmSh': '7051900', 'ncmSh': 'Outras alfaces frescas ou refrigeradas', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '3,79', 'valor': '3,79'}, {'numeroProduto': '15', 'descricaoProdutoServico': 'Maionese 500g Pet/ Light Hellmanns UN', 'codigoNcmSh': '21039011', 'ncmSh': 'Maionese, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '8,54', 'valor': '8,54'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Po P/ Gelatina 24g Sem Sabor Bretzke UN', 'codigoNcmSh': '35030019', 'ncmSh': 'Outras gelatinas e seus derivados', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '4,74', 'valor': '4,74'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Manjericao 15g Kamila UN', 'codigoNcmSh': '12119090', 'ncmSh': 'Outras plantas e partes, para perfumaria, medicina e semelhantes', 'cfop': '5929', 'quantidade': '1,00', 'unidade': 'UN', 'valorUnitario': '1,89', 'valor': '1,89'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 104373, 'codigoOrgaoSuperiorDestinatario': '52000', 'orgaoSuperiorDestinatario': 'Ministério da Defesa', 'codigoOrgaoDestinatario': '52111', 'orgaoDestinatario': 'Comando da Aeronáutica', 'nomeFornecedor': 'SUNNY ALIMENTACAO E SERVICOSLTDA', 'cnpjFornecedor': '10.393.465/0001-03', 'municipioFornecedor': 'SAO PAULO', 'chaveNotaFiscal': '35211210393465000103550010000027441992520231', 'valorNotaFiscal': '13.200,00', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '03/12/2021 14:24:14', 'dataEmissao': '03/12/2021', 'numero': 2744, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '3', 'descricaoProdutoServico': 'Acucar 1KG', 'codigoNcmSh': '17019900', 'ncmSh': 'Outros açúcares de cana, beterraba, sacarose químicamente pura, sol.', 'cfop': '5405', 'quantidade': '200,00', 'unidade': 'PCT', 'valorUnitario': '3,54', 'valor': '708,00'}, {'numeroProduto': '15', 'descricaoProdutoServico': 'Biscoito Waffer 120G', 'codigoNcmSh': '19059020', 'ncmSh': 'Bolachas', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'PCT', 'valorUnitario': '1,54', 'valor': '123,20'}, {'numeroProduto': '17', 'descricaoProdutoServico': 'Gelatina 25G', 'codigoNcmSh': '21069029', 'ncmSh': 'Pós para preparações de cremes, sorvetes, flans, gelatinas ou preparações similares', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'SC', 'valorUnitario': '0,75', 'valor': '120,00'}, {'numeroProduto': '30', 'descricaoProdutoServico': 'Sabao em Pedra com 05', 'codigoNcmSh': '34011900', 'ncmSh': 'Outros sabões/produtos/preparações, em barras, pedaços, etc.', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'PCT', 'valorUnitario': '4,20', 'valor': '168,00'}, {'numeroProduto': '27', 'descricaoProdutoServico': 'Creme de Leite 200G', 'codigoNcmSh': '4015021', 'ncmSh': 'Creme de leite UHT (Ultra High Temperature), com um teor, em peso, de matérias gordas, superior a 10 %, não concentrados nem adicionados de açúcar ou de outros edulcorantes', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '2,99', 'valor': '239,20'}, {'numeroProduto': '20', 'descricaoProdutoServico': 'Milho lata 170G', 'codigoNcmSh': '20058000', 'ncmSh': 'Milho doce (Zea mays var. saccharata), preparados ou conservados, exceto em vinagre ou em ácido acético, não congelados', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '2,53', 'valor': '202,40'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Trigo 1KG', 'codigoNcmSh': '11010010', 'ncmSh': 'Farinha de trigo', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'PCT', 'valorUnitario': '3,48', 'valor': '139,20'}, {'numeroProduto': '32', 'descricaoProdutoServico': 'Esponja de aco com 08', 'codigoNcmSh': '73231000', 'ncmSh': 'Palha de ferro ou aço; esponjas, esfregões, luvas e artefatos semelhantes para limpeza, polimento ou usos semelhantes', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'PCT', 'valorUnitario': '2,30', 'valor': '92,00'}, {'numeroProduto': '36', 'descricaoProdutoServico': 'Pasta de dente 90G', 'codigoNcmSh': '33061000', 'ncmSh': 'Dentifrícios', 'cfop': '5405', 'quantidade': '200,00', 'unidade': 'UN', 'valorUnitario': '2,52', 'valor': '504,00'}, {'numeroProduto': '19', 'descricaoProdutoServico': 'Espaguete 500G', 'codigoNcmSh': '19021100', 'ncmSh': 'Massas alimentícias não cozidas, nem recheadas, nem preparadas de outro modo, que contenham ovos', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'PCT', 'valorUnitario': '2,24', 'valor': '358,40'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Composto Lacteo 200G', 'codigoNcmSh': '4049000', 'ncmSh': 'Outros produtos constituídos do leite, mesmo adocicados, etc', 'cfop': '5102', 'quantidade': '320,00', 'unidade': 'PCT', 'valorUnitario': '2,78', 'valor': '889,60'}, {'numeroProduto': '23', 'descricaoProdutoServico': 'Azeite de oliva  200ML', 'codigoNcmSh': '15079011', 'ncmSh': 'Óleo de soja, refinado, em recipientes com capacidade inferior ou igual a 5 litros', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'LT', 'valorUnitario': '3,79', 'valor': '151,60'}, {'numeroProduto': '26', 'descricaoProdutoServico': 'Leite Condensado 395G', 'codigoNcmSh': '4029900', 'ncmSh': 'Outros leites, cremes de leite, concentrados, adocicados', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '3,78', 'valor': '151,20'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Biscoito Salgado 200G', 'codigoNcmSh': '19053100', 'ncmSh': 'Bolachas e biscoitos, adicionados de edulcorante', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'PCT', 'valorUnitario': '1,98', 'valor': '316,80'}, {'numeroProduto': '31', 'descricaoProdutoServico': 'Papel higienico 4 rolos', 'codigoNcmSh': '48181000', 'ncmSh': 'Papel higiênico', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'PCT', 'valorUnitario': '4,69', 'valor': '187,60'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Feijao Carioca Tipo 1 1KG', 'codigoNcmSh': '7133399', 'ncmSh': 'Outros feijões comuns, secos, em grãos', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'PCT', 'valorUnitario': '5,85', 'valor': '936,00'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Extrato de Tomate 340G', 'codigoNcmSh': '21032010', 'ncmSh': 'Ketchup e outros molhos de tomate, preparado, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'SC', 'valorUnitario': '2,16', 'valor': '172,80'}, {'numeroProduto': '38', 'descricaoProdutoServico': 'Shampoo 325ML', 'codigoNcmSh': '33051000', 'ncmSh': 'Xampus para os cabelos', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '9,10', 'valor': '364,00'}, {'numeroProduto': '33', 'descricaoProdutoServico': 'Detergente 500ML', 'codigoNcmSh': '34022000', 'ncmSh': 'Preparações tensoativas, para lavagem e limpeza', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '1,78', 'valor': '142,40'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Charque 500G', 'codigoNcmSh': '2102000', 'ncmSh': 'Carnes de bovinos, salgadas/em salmoura/secas/defumadas', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'PC', 'valorUnitario': '21,61', 'valor': '864,40'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Achocolatado 200G', 'codigoNcmSh': '18069000', 'ncmSh': 'Outros chocolates e preparações alimentícias contendo cacau', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'PCT', 'valorUnitario': '3,23', 'valor': '258,40'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Farofa Mandioca 250G', 'codigoNcmSh': '11062000', 'ncmSh': 'Farinhas, sêmolas e pós, de sagu ou das raízes ou tubérculos, da posição 07.14', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'PCT', 'valorUnitario': '2,20', 'valor': '352,00'}, {'numeroProduto': '42', 'descricaoProdutoServico': 'LENTINHA', 'codigoNcmSh': '7134090', 'ncmSh': 'Outras lentilhas secas, em grãos', 'cfop': '5102', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '2,86', 'valor': '228,80'}, {'numeroProduto': '29', 'descricaoProdutoServico': 'Salcicha lata 180G', 'codigoNcmSh': '16010000', 'ncmSh': 'Enchidos e produtos semelhantes, de carne, de miudezas ou de sangue; preparações alimentícias à base de tais produtos', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'LT', 'valorUnitario': '2,00', 'valor': '160,00'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Sardinha 125G', 'codigoNcmSh': '16041310', 'ncmSh': 'Preparações e conservas, de sardinhas, inteiros ou em pedaços, exceto peixes picados', 'cfop': '5405', 'quantidade': '120,00', 'unidade': 'LT', 'valorUnitario': '3,40', 'valor': '408,00'}, {'numeroProduto': '44', 'descricaoProdutoServico': 'CAIXA PAPELAO', 'codigoNcmSh': '39231090', 'ncmSh': 'Outros artigos semelhantes a caixas, engradados, etc, de plástico', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '0,50', 'valor': '20,00'}, {'numeroProduto': '35', 'descricaoProdutoServico': 'Desinfetante 500ML', 'codigoNcmSh': '38089429', 'ncmSh': 'Outros desinfetantes apresentados de outro modo', 'cfop': '5102', 'quantidade': '80,00', 'unidade': 'LT', 'valorUnitario': '2,36', 'valor': '188,80'}, {'numeroProduto': '40', 'descricaoProdutoServico': 'Esponja Multiuso', 'codigoNcmSh': '73231000', 'ncmSh': 'Palha de ferro ou aço; esponjas, esfregões, luvas e artefatos semelhantes para limpeza, polimento ou usos semelhantes', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '1,78', 'valor': '71,20'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Arroz Tipo 1 5KG', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'PCT', 'valorUnitario': '22,00', 'valor': '880,00'}, {'numeroProduto': '37', 'descricaoProdutoServico': 'Sabao em po 1KG', 'codigoNcmSh': '34022000', 'ncmSh': 'Preparações tensoativas, para lavagem e limpeza', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'PCT', 'valorUnitario': '3,78', 'valor': '302,40'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Cafe 500G', 'codigoNcmSh': '9012100', 'ncmSh': 'Café torrado, não descafeinado', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'PCT', 'valorUnitario': '5,55', 'valor': '444,00'}, {'numeroProduto': '25', 'descricaoProdutoServico': 'Vinagre 750ML', 'codigoNcmSh': '22090000', 'ncmSh': 'Vinagres e seus sucedâneos obtidos a partir do ácido acético, para usos alimentares', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'LT', 'valorUnitario': '1,00', 'valor': '40,00'}, {'numeroProduto': '22', 'descricaoProdutoServico': 'Fuba 500G', 'codigoNcmSh': '11022000', 'ncmSh': 'Farinha de milho', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'PCT', 'valorUnitario': '1,35', 'valor': '216,00'}, {'numeroProduto': '43', 'descricaoProdutoServico': 'SAL REFINADO', 'codigoNcmSh': '25010020', 'ncmSh': 'Sal de mesa', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'KG', 'valorUnitario': '2,10', 'valor': '84,00'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Amido 200G', 'codigoNcmSh': '11081200', 'ncmSh': 'Amido de milho', 'cfop': '5102', 'quantidade': '200,00', 'unidade': 'SC', 'valorUnitario': '2,03', 'valor': '406,00'}, {'numeroProduto': '16', 'descricaoProdutoServico': 'Goiabada 300G', 'codigoNcmSh': '20071000', 'ncmSh': 'Preparações homogeneizadas de frutas, obtidos por cozimento, com ou sem adição de açúcar ou de outros edulcorantes', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '2,76', 'valor': '220,80'}, {'numeroProduto': '18', 'descricaoProdutoServico': 'Suco concentrado 500ML', 'codigoNcmSh': '20096900', 'ncmSh': 'Outros sucos de uvas', 'cfop': '5102', 'quantidade': '80,00', 'unidade': 'LT', 'valorUnitario': '2,59', 'valor': '207,20'}, {'numeroProduto': '34', 'descricaoProdutoServico': 'Agua Sanitaria 1LT', 'codigoNcmSh': '28289011', 'ncmSh': 'Hipocloritos de sódio', 'cfop': '5405', 'quantidade': '80,00', 'unidade': 'LT', 'valorUnitario': '2,20', 'valor': '176,00'}, {'numeroProduto': '41', 'descricaoProdutoServico': 'EMBALAGEM PLASTICA COM ALCA AMARELA', 'codigoNcmSh': '39232190', 'ncmSh': 'Outros sacos, bolsas e cartuchos, de polímeros de etileno', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '1,55', 'valor': '62,00'}, {'numeroProduto': '24', 'descricaoProdutoServico': 'Aveia 200G', 'codigoNcmSh': '11041200', 'ncmSh': 'Grãos de aveia, esmagados ou em flocos', 'cfop': '5102', 'quantidade': '80,00', 'unidade': 'UN', 'valorUnitario': '3,85', 'valor': '308,00'}, {'numeroProduto': '45', 'descricaoProdutoServico': 'Oleo de Soja GFA 900 ML', 'codigoNcmSh': '15079011', 'ncmSh': 'Óleo de soja, refinado, em recipientes com capacidade inferior ou igual a 5 litros', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '10,45', 'valor': '418,00'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Bisoito Maisena 200G', 'codigoNcmSh': '19053100', 'ncmSh': 'Bolachas e biscoitos, adicionados de edulcorante', 'cfop': '5102', 'quantidade': '80,00', 'unidade': 'PCT', 'valorUnitario': '2,02', 'valor': '161,60'}, {'numeroProduto': '28', 'descricaoProdutoServico': 'Tempero Completo 300G', 'codigoNcmSh': '21039021', 'ncmSh': 'Condimentos e temperos, compostos, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'SC', 'valorUnitario': '4,00', 'valor': '160,00'}, {'numeroProduto': '21', 'descricaoProdutoServico': 'Maionese 200G', 'codigoNcmSh': '21039011', 'ncmSh': 'Maionese, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5405', 'quantidade': '120,00', 'unidade': 'SC', 'valorUnitario': '1,93', 'valor': '231,60'}, {'numeroProduto': '39', 'descricaoProdutoServico': 'Condicionador 325ML', 'codigoNcmSh': '33059000', 'ncmSh': 'Outras preparações capilares', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'UN', 'valorUnitario': '9,11', 'valor': '364,40'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 106015, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36000', 'orgaoDestinatario': 'Ministério da Saúde - Unidades com vínculo direto', 'nomeFornecedor': 'NUTRIXX SUPLEMENTOS ALIMENTARES LTDA', 'cnpjFornecedor': '12.401.269/0001-69', 'municipioFornecedor': 'CASTANHAL', 'chaveNotaFiscal': '15220112401269000169550010000056231002227840', 'valorNotaFiscal': '3.593,49', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '25/01/2022 12:15:11', 'dataEmissao': '25/01/2022', 'numero': 5623, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '3', 'descricaoProdutoServico': 'NUTREN JUNIOR BAUNILHA 24X400G', 'codigoNcmSh': '21069090', 'ncmSh': 'Outras preparações alimentícias', 'cfop': '5102', 'quantidade': '12,00', 'unidade': 'UN', 'valorUnitario': '40,00', 'valor': '480,00'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'MUCILON LATA Arroz 18x400G', 'codigoNcmSh': '19011030', 'ncmSh': 'Preparações à base de farinha, grumos, sêmola ou amido, para alimentação de crianças, acondicionadas para venda a retalho', 'cfop': '5405', 'quantidade': '50,00', 'unidade': 'UN', 'valorUnitario': '7,88', 'valor': '394,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'NUTREN JUNIOR BAUNILHA 24X400G', 'codigoNcmSh': '21069090', 'ncmSh': 'Outras preparações alimentícias', 'cfop': '5102', 'quantidade': '7,00', 'unidade': 'UN', 'valorUnitario': '40,00', 'valor': '280,00'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'MUCILON LATA Multicereais 18x400G', 'codigoNcmSh': '19011030', 'ncmSh': 'Preparações à base de farinha, grumos, sêmola ou amido, para alimentação de crianças, acondicionadas para venda a retalho', 'cfop': '5405', 'quantidade': '50,00', 'unidade': 'UN', 'valorUnitario': '9,90', 'valor': '495,00'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'NUTREN SENIOR PO SEM SABOR 24X370G', 'codigoNcmSh': '19019090', 'ncmSh': 'Outras preparações alimentícias de farinhas, etc, cacau < 40%', 'cfop': '5102', 'quantidade': '43,00', 'unidade': 'UN', 'valorUnitario': '42,43', 'valor': '1.824,49'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'NUTREN JUNIOR BAUNILHA 24X400G', 'codigoNcmSh': '21069090', 'ncmSh': 'Outras preparações alimentícias', 'cfop': '5102', 'quantidade': '3,00', 'unidade': 'UN', 'valorUnitario': '40,00', 'valor': '120,00'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 603681, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36201', 'orgaoDestinatario': 'Fundação Oswaldo Cruz', 'nomeFornecedor': 'FELIPE DE LIMA FIRMINO', 'cnpjFornecedor': '35.119.484/0001-77', 'municipioFornecedor': 'RIO DE JANEIRO', 'chaveNotaFiscal': '33211135119484000177550010000001041703585865', 'valorNotaFiscal': '675,95', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '24/11/2021 07:07:10', 'dataEmissao': '23/11/2021', 'numero': 104, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '5', 'descricaoProdutoServico': 'Amido de Milho Maizena', 'codigoNcmSh': '11081200', 'ncmSh': 'Amido de milho', 'cfop': '5102', 'quantidade': '15,00', 'unidade': 'un', 'valorUnitario': '8,00', 'valor': '120,00'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Acucar KG', 'codigoNcmSh': '17011300', 'ncmSh': 'Açúcar de cana mencionado na nota 2 da posição 1701', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'un', 'valorUnitario': '4,36', 'valor': '43,60'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'MUCILON BL Milho Sachet 230g BR', 'codigoNcmSh': '19011030', 'ncmSh': 'Preparações à base de farinha, grumos, sêmola ou amido, para alimentação de crianças, acondicionadas para venda a retalho', 'cfop': '5102', 'quantidade': '5,00', 'unidade': 'un', 'valorUnitario': '6,97', 'valor': '34,85'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'MUCILON BL Arroz Sachet 230g BR', 'codigoNcmSh': '19011030', 'ncmSh': 'Preparações à base de farinha, grumos, sêmola ou amido, para alimentação de crianças, acondicionadas para venda a retalho', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'un', 'valorUnitario': '8,00', 'valor': '80,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'MUCILON BL Multicereais Sachet  230gBR', 'codigoNcmSh': '19011030', 'ncmSh': 'Preparações à base de farinha, grumos, sêmola ou amido, para alimentação de crianças, acondicionadas para venda a retalho', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'un', 'valorUnitario': '7,71', 'valor': '77,10'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Leite em po 400G IFF', 'codigoNcmSh': '4022110', 'ncmSh': 'Leite integral, em pó, com um teor, em peso, de matérias gordas, superior a 1,5 %, sem adição de açúcar ou de outros edulcorantes', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'un', 'valorUnitario': '16,02', 'valor': '320,40'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 805259, 'codigoOrgaoSuperiorDestinatario': '26000', 'orgaoSuperiorDestinatario': 'Ministério da Educação', 'codigoOrgaoDestinatario': '26408', 'orgaoDestinatario': 'Instituto Federal do Maranhão', 'nomeFornecedor': 'COOPERATIVA AGROCOLOGICA PELA VIDA DO CERRADO SUL MARANHENSE', 'cnpjFornecedor': '41.385.055/0001-44', 'municipioFornecedor': 'SAO RAIMUNDO DAS MANGABEIRAS', 'chaveNotaFiscal': '21211241385055000144550010000006861000001468', 'valorNotaFiscal': '129.799,53', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '20/12/2021 17:23:08', 'dataEmissao': '20/12/2021', 'numero': 686, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '5', 'descricaoProdutoServico': 'Feijao', 'codigoNcmSh': '7133321', 'ncmSh': 'Feijão comum, branco, seco, para semeadura', 'cfop': '5102', 'quantidade': '3.172,00', 'unidade': 'KG', 'valorUnitario': '10,00', 'valor': '31.720,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Farinha Branca Seca', 'codigoNcmSh': '11022000', 'ncmSh': 'Farinha de milho', 'cfop': '5102', 'quantidade': '1.586,00', 'unidade': 'UN', 'valorUnitario': '6,50', 'valor': '10.309,00'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Polpa de Goiaba', 'codigoNcmSh': '8119000', 'ncmSh': 'Outras frutas não cozidas ou cozidas em água ou vapor, congeladas, mesmo adicionadas de açúcar ou de outros edulcorantes', 'cfop': '5102', 'quantidade': '1.586,00', 'unidade': 'KG', 'valorUnitario': '10,30', 'valor': '16.335,80'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Polpa de Caju', 'codigoNcmSh': '8119000', 'ncmSh': 'Outras frutas não cozidas ou cozidas em água ou vapor, congeladas, mesmo adicionadas de açúcar ou de outros edulcorantes', 'cfop': '5102', 'quantidade': '223,00', 'unidade': 'KG', 'valorUnitario': '10,00', 'valor': '2.230,00'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Raiz de Mandioca (macaxeira)', 'codigoNcmSh': '7141000', 'ncmSh': 'Raízes de mandioca, frescas, refrigeradas, congeladas ou secas', 'cfop': '5102', 'quantidade': '1.181,00', 'unidade': 'KG', 'valorUnitario': '4,15', 'valor': '4.901,15'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Mamao Papaya', 'codigoNcmSh': '8072000', 'ncmSh': 'Mamões (papaias) frescos', 'cfop': '5102', 'quantidade': '1.801,60', 'unidade': 'KG', 'valorUnitario': '3,42', 'valor': '6.161,47'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Abobora', 'codigoNcmSh': '7099300', 'ncmSh': 'Abóboras, abobrinhas e cabaças, fresca ou refrigerada', 'cfop': '5102', 'quantidade': '2.478,80', 'unidade': 'KG', 'valorUnitario': '4,05', 'valor': '10.039,14'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Polpa de Acerola', 'codigoNcmSh': '8119000', 'ncmSh': 'Outras frutas não cozidas ou cozidas em água ou vapor, congeladas, mesmo adicionadas de açúcar ou de outros edulcorantes', 'cfop': '5102', 'quantidade': '223,00', 'unidade': 'KG', 'valorUnitario': '10,00', 'valor': '2.230,00'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Polpa de caja', 'codigoNcmSh': '8119000', 'ncmSh': 'Outras frutas não cozidas ou cozidas em água ou vapor, congeladas, mesmo adicionadas de açúcar ou de outros edulcorantes', 'cfop': '5102', 'quantidade': '1.140,00', 'unidade': 'KG', 'valorUnitario': '13,30', 'valor': '15.162,00'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Arroz', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '3.965,00', 'unidade': 'KG', 'valorUnitario': '5,65', 'valor': '22.402,25'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Banana', 'codigoNcmSh': '8031000', 'ncmSh': 'Bananas-da-terra, frescas ou secas', 'cfop': '5102', 'quantidade': '2.002,10', 'unidade': 'KG', 'valorUnitario': '4,15', 'valor': '8.308,72'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 1106575, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36201', 'orgaoDestinatario': 'Fundação Oswaldo Cruz', 'nomeFornecedor': 'WS DISTRIBUIDORA ALIMENTOS LTDA-EPP', 'cnpjFornecedor': '01.220.638/0001-09', 'municipioFornecedor': 'RIO DE JANEIRO', 'chaveNotaFiscal': '33220201220638000109550010000278731000830087', 'valorNotaFiscal': '460,21', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '16/02/2022 11:53:34', 'dataEmissao': '16/02/2022', 'numero': 27873, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '3', 'descricaoProdutoServico': 'Arroz Branco', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '15,00', 'unidade': 'Kg', 'valorUnitario': '3,88', 'valor': '58,20'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Chocolate em po 200gr 004', 'codigoNcmSh': '18063210', 'ncmSh': 'Chocolate não recheado, em tabletes, barras e paus', 'cfop': '5405', 'quantidade': '12,00', 'unidade': 'Cx', 'valorUnitario': '24,55', 'valor': '294,60'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Aveia 250gr', 'codigoNcmSh': '11042200', 'ncmSh': 'Outros grãos trabalhados (por exemplo, descascados, em pérolas, cortados ou partidos), de aveia', 'cfop': '5102', 'quantidade': '3,00', 'unidade': 'Cx', 'valorUnitario': '3,75', 'valor': '11,25'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Achocolatado 593', 'codigoNcmSh': '18069000', 'ncmSh': 'Outros chocolates e preparações alimentícias contendo cacau', 'cfop': '5405', 'quantidade': '16,00', 'unidade': 'Un', 'valorUnitario': '6,01', 'valor': '96,16'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 1504873, 'codigoOrgaoSuperiorDestinatario': '26000', 'orgaoSuperiorDestinatario': 'Ministério da Educação', 'codigoOrgaoDestinatario': '26416', 'orgaoDestinatario': 'Instituto Federal do Pará', 'nomeFornecedor': 'ACS DA ENCARNACAO EIRELI', 'cnpjFornecedor': '22.326.788/0001-63', 'municipioFornecedor': 'TUCURUI', 'chaveNotaFiscal': '15211222326788000163550010000002991011114311', 'valorNotaFiscal': '8.351,14', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '14/12/2021 09:45:15', 'dataEmissao': '14/12/2021', 'numero': 299, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '2', 'descricaoProdutoServico': 'Arroz tipo 1', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '540,00', 'unidade': 'KG', 'valorUnitario': '4,18', 'valor': '2.257,20'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'copo descartavel', 'codigoNcmSh': '39241000', 'ncmSh': 'Serviços de mesa e outros utensílios de mesa ou de cozinha, de plásticos', 'cfop': '5102', 'quantidade': '30,00', 'unidade': 'PT', 'valorUnitario': '4,00', 'valor': '120,00'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Biscoito salgado integral', 'codigoNcmSh': '19059020', 'ncmSh': 'Bolachas', 'cfop': '5102', 'quantidade': '30,00', 'unidade': 'UN', 'valorUnitario': '9,80', 'valor': '294,00'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'pano de chao', 'codigoNcmSh': '63071000', 'ncmSh': 'Rodilhas, esfregões, panos de prato ou de cozinha, flanelas e artefatos de limpeza semelhantes', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'UN', 'valorUnitario': '4,98', 'valor': '99,60'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Pimenta do reino preta ptc 50g', 'codigoNcmSh': '9041200', 'ncmSh': 'Pimenta (do gênero Piper), triturada ou em pó', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'PT', 'valorUnitario': '3,44', 'valor': '68,80'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Alho in natura', 'codigoNcmSh': '7032090', 'ncmSh': 'Alhos, frescos ou refrigerados, exceto para semeadura', 'cfop': '5102', 'quantidade': '50,00', 'unidade': 'KG', 'valorUnitario': '15,48', 'valor': '774,00'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Macarrao de Arroz', 'codigoNcmSh': '19021900', 'ncmSh': 'Outras massas alimentícias não cozidas, nem recheadas, nem preparadas de outro modo', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'KG', 'valorUnitario': '6,50', 'valor': '65,00'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Linguca tipo Calabresa', 'codigoNcmSh': '16010000', 'ncmSh': 'Enchidos e produtos semelhantes, de carne, de miudezas ou de sangue; preparações alimentícias à base de tais produtos', 'cfop': '5102', 'quantidade': '70,00', 'unidade': 'KG', 'valorUnitario': '14,50', 'valor': '1.015,00'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'polpa de fruta acerola 1kg', 'codigoNcmSh': '20089900', 'ncmSh': 'Outras frutas, partes de plantas, preparadas/conservadas de outro modo', 'cfop': '5102', 'quantidade': '50,00', 'unidade': 'KG', 'valorUnitario': '14,18', 'valor': '709,00'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Massa seca p Lasanha', 'codigoNcmSh': '19021900', 'ncmSh': 'Outras massas alimentícias não cozidas, nem recheadas, nem preparadas de outro modo', 'cfop': '5102', 'quantidade': '6,00', 'unidade': 'KG', 'valorUnitario': '10,99', 'valor': '65,94'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Batata Doce in Natura', 'codigoNcmSh': '7142000', 'ncmSh': 'Batatas-doces, frescas, refrigeradas, congeladas ou secas', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '4,95', 'valor': '1.980,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Beterraba in Natura', 'codigoNcmSh': '7069000', 'ncmSh': 'Beterrabas, rabanetes e outras raízes, frescas, refrigeradas', 'cfop': '5102', 'quantidade': '70,00', 'unidade': 'KG', 'valorUnitario': '5,30', 'valor': '371,00'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Tangerina', 'codigoNcmSh': '8052100', 'ncmSh': 'Mandarinas (incluindo as tangerinas e as satsumas)', 'cfop': '5102', 'quantidade': '65,00', 'unidade': 'KG', 'valorUnitario': '5,80', 'valor': '377,00'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'oregano pct 250g', 'codigoNcmSh': '12119010', 'ncmSh': 'Orégano (Origanum vulgare) fresco ou seco, para perfumaria, medicina, etc.', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'PT', 'valorUnitario': '7,73', 'valor': '154,60'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 2102643, 'codigoOrgaoSuperiorDestinatario': '26000', 'orgaoSuperiorDestinatario': 'Ministério da Educação', 'codigoOrgaoDestinatario': '26420', 'orgaoDestinatario': 'Instituto Federal Farroupilha', 'nomeFornecedor': 'COOP PROD RURAIS AGRIC FAMILIAR DE FW', 'cnpjFornecedor': '10.378.011/0001-63', 'municipioFornecedor': 'FREDERICO WESTPHALEN', 'chaveNotaFiscal': '43211110378011000163550000000021251891595167', 'valorNotaFiscal': '5.051,20', 'tipoEventoMaisRecente': 'Cancelamento da NF-e', 'dataTipoEventoMaisRecente': '10/11/2021 13:59:16', 'dataEmissao': '09/11/2021', 'numero': 2125, 'serie': 0}, 'itensNotaFiscal': [{'numeroProduto': '3', 'descricaoProdutoServico': 'Abacaxi', 'codigoNcmSh': '8043000', 'ncmSh': 'Abacaxis frescos ou secos', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'UN', 'valorUnitario': '5,00', 'valor': '800,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Maca', 'codigoNcmSh': '8081000', 'ncmSh': 'Maçãs frescas', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'KG', 'valorUnitario': '5,80', 'valor': '928,00'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Mamao', 'codigoNcmSh': '8072000', 'ncmSh': 'Mamões (papaias) frescos', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'KG', 'valorUnitario': '6,00', 'valor': '960,00'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Banana Prata', 'codigoNcmSh': '8031000', 'ncmSh': 'Bananas-da-terra, frescas ou secas', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'KG', 'valorUnitario': '4,77', 'valor': '763,20'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Arroz Branco 2k', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '160,00', 'unidade': 'UN', 'valorUnitario': '10,00', 'valor': '1.600,00'}], 'eventosNotaFiscal': [{'dataEvento': '10/11/2021 13:59:16', 'tipoEvento': 'Cancelamento', 'evento': 'Protocolo: 143210233388248', 'motivo': 'emissao para transporte'}]},{'notaFiscalDTO': {'id': 2105317, 'codigoOrgaoSuperiorDestinatario': '26000', 'orgaoSuperiorDestinatario': 'Ministério da Educação', 'codigoOrgaoDestinatario': '26420', 'orgaoDestinatario': 'Instituto Federal Farroupilha', 'nomeFornecedor': 'COOP PROD RURAIS AGRIC FAMILIAR DE FW', 'cnpjFornecedor': '10.378.011/0001-63', 'municipioFornecedor': 'FREDERICO WESTPHALEN', 'chaveNotaFiscal': '43211210378011000163550000000022731541755889', 'valorNotaFiscal': '114.084,00', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '30/12/2021 11:32:22', 'dataEmissao': '30/12/2021', 'numero': 2273, 'serie': 0}, 'itensNotaFiscal': [{'numeroProduto': '21', 'descricaoProdutoServico': 'Pao Caseiro Und', 'codigoNcmSh': '19059010', 'ncmSh': 'Pão de forma', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '6,36', 'valor': '2.544,00'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Banana Prata', 'codigoNcmSh': '8031000', 'ncmSh': 'Bananas-da-terra, frescas ou secas', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'KG', 'valorUnitario': '3,96', 'valor': '3.168,00'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Acucara Mascavo Piaia', 'codigoNcmSh': '17019900', 'ncmSh': 'Outros açúcares de cana, beterraba, sacarose químicamente pura, sol.', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '10,08', 'valor': '4.032,00'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Queijo Colonial Eduvavi', 'codigoNcmSh': '4061010', 'ncmSh': 'Queijo tipo mussarela, fresco (não curado)', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '33,93', 'valor': '13.572,00'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Milho Pipoca', 'codigoNcmSh': '10059010', 'ncmSh': 'Milho em grão, exceto para semeadura', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '3,79', 'valor': '1.516,00'}, {'numeroProduto': '19', 'descricaoProdutoServico': 'Salame Bisolo', 'codigoNcmSh': '16010000', 'ncmSh': 'Enchidos e produtos semelhantes, de carne, de miudezas ou de sangue; preparações alimentícias à base de tais produtos', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '14,99', 'valor': '5.996,00'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Geleia Sabor do Vale 400g', 'codigoNcmSh': '20079910', 'ncmSh': 'Geleias e marmelades, de outras frutas', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '7,19', 'valor': '2.876,00'}, {'numeroProduto': '16', 'descricaoProdutoServico': 'Bacon Defumado', 'codigoNcmSh': '2101200', 'ncmSh': 'Barrigas e peitos, entremeados, de suíno, salgados, etc.', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '14,36', 'valor': '5.744,00'}, {'numeroProduto': '15', 'descricaoProdutoServico': 'Carne Suina Costeliha Defumada', 'codigoNcmSh': '2101900', 'ncmSh': 'Outras carnes de suíno, salgadas ou em salmoura, secas, etc.', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '35,93', 'valor': '14.372,00'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Suco de Uva Integral Vitis 1,5l', 'codigoNcmSh': '20096100', 'ncmSh': 'Suco (sumo) de uva (incluindo os mostos de uvas), com valor Brix não superior a 30', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '17,66', 'valor': '7.064,00'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Feijao Preto', 'codigoNcmSh': '20055100', 'ncmSh': 'Feijões em grãos, preparados ou conservados, exceto em vinagre ou em ácido acético, não congelados', 'cfop': '5102', 'quantidade': '1.200,00', 'unidade': 'KG', 'valorUnitario': '8,93', 'valor': '10.716,00'}, {'numeroProduto': '17', 'descricaoProdutoServico': 'Arroz Branco 5kg', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '27,09', 'valor': '10.836,00'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Laranja Tipo Suco', 'codigoNcmSh': '8051000', 'ncmSh': 'Laranjas, frescas ou secas', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'KG', 'valorUnitario': '2,88', 'valor': '2.304,00'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Canjica Amarela Progresso 500g', 'codigoNcmSh': '32041913', 'ncmSh': 'Outras preparações próprias para colorir alimentos', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'PCTE', 'valorUnitario': '3,58', 'valor': '1.432,00'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Bolacha de Polvilho', 'codigoNcmSh': '19059020', 'ncmSh': 'Bolachas', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '5,83', 'valor': '2.332,00'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Mel Rainha Alimentos 500 G', 'codigoNcmSh': '4090000', 'ncmSh': 'Mel natural', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'UN', 'valorUnitario': '13,66', 'valor': '5.464,00'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Farinha de Milho Di Bernardo', 'codigoNcmSh': '11022000', 'ncmSh': 'Farinha de milho', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'KG', 'valorUnitario': '4,49', 'valor': '3.592,00'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Ovos Tradicao Caipira', 'codigoNcmSh': '19021100', 'ncmSh': 'Massas alimentícias não cozidas, nem recheadas, nem preparadas de outro modo, que contenham ovos', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'DUZIA', 'valorUnitario': '8,26', 'valor': '6.608,00'}, {'numeroProduto': '18', 'descricaoProdutoServico': 'Bolacha Caseira de Acucar Mascavo', 'codigoNcmSh': '19059020', 'ncmSh': 'Bolachas', 'cfop': '5102', 'quantidade': '400,00', 'unidade': 'KG', 'valorUnitario': '5,83', 'valor': '2.332,00'}, {'numeroProduto': '20', 'descricaoProdutoServico': 'Brocolis', 'codigoNcmSh': '7041000', 'ncmSh': 'Couve-flor e brócolis, frescos ou refrigerados', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'UN', 'valorUnitario': '5,50', 'valor': '4.400,00'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Batata Doce', 'codigoNcmSh': '7061000', 'ncmSh': 'Cenouras e nabos, frescos ou refrigerados', 'cfop': '5102', 'quantidade': '800,00', 'unidade': 'KG', 'valorUnitario': '3,98', 'valor': '3.184,00'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 2204897, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36201', 'orgaoDestinatario': 'Fundação Oswaldo Cruz', 'nomeFornecedor': 'WS DISTRIBUIDORA ALIMENTOS LTDA-EPP', 'cnpjFornecedor': '01.220.638/0001-09', 'municipioFornecedor': 'RIO DE JANEIRO', 'chaveNotaFiscal': '33211201220638000109550010000273201000824559', 'valorNotaFiscal': '1.699,44', 'tipoEventoMaisRecente': 'Manifestação do destinatário - Ciência da operação', 'dataTipoEventoMaisRecente': '15/12/2021 15:33:24', 'dataEmissao': '15/12/2021', 'numero': 27320, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '1', 'descricaoProdutoServico': 'Creme de leite A21', 'codigoNcmSh': '4029900', 'ncmSh': 'Outros leites, cremes de leite, concentrados, adocicados', 'cfop': '5405', 'quantidade': '54,00', 'unidade': 'Und', 'valorUnitario': '5,24', 'valor': '282,96'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Feijao branco 500gr', 'codigoNcmSh': '7133329', 'ncmSh': 'Outros feijões comuns, brancos, secos, em grãos', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'Pct', 'valorUnitario': '6,51', 'valor': '130,20'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Feijao preto', 'codigoNcmSh': '7133319', 'ncmSh': 'Outros feijões comuns, pretos, secos, em grãos', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'Kg', 'valorUnitario': '7,22', 'valor': '72,20'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Arroz Branco', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '30,00', 'unidade': 'Kg', 'valorUnitario': '3,88', 'valor': '116,40'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Queijo creme cheese 174', 'codigoNcmSh': '4061090', 'ncmSh': 'Outros queijos frescos (não curados), inclusive requeijão, etc.', 'cfop': '5102', 'quantidade': '40,00', 'unidade': 'Un', 'valorUnitario': '9,90', 'valor': '396,00'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Catchup 200gr', 'codigoNcmSh': '21032010', 'ncmSh': 'Ketchup e outros molhos de tomate, preparado, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'Un', 'valorUnitario': '2,68', 'valor': '26,80'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Mostarda', 'codigoNcmSh': '21033021', 'ncmSh': 'Mostarda preparada, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5405', 'quantidade': '10,00', 'unidade': 'Un', 'valorUnitario': '3,21', 'valor': '32,10'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Molho de tomate 340gr', 'codigoNcmSh': '20029090', 'ncmSh': 'Outros tomates preparados ou conservados, exceto em vinagre ou em ácido acético', 'cfop': '5102', 'quantidade': '48,00', 'unidade': 'Un', 'valorUnitario': '2,41', 'valor': '115,68'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Gelatina de limao a21 .', 'codigoNcmSh': '21069029', 'ncmSh': 'Pós para preparações de cremes, sorvetes, flans, gelatinas ou preparações similares', 'cfop': '5405', 'quantidade': '40,00', 'unidade': 'Un', 'valorUnitario': '1,28', 'valor': '51,20'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'ID-59598 Extrato de tomate', 'codigoNcmSh': '20029090', 'ncmSh': 'Outros tomates preparados ou conservados, exceto em vinagre ou em ácido acético', 'cfop': '5405', 'quantidade': '48,00', 'unidade': 'Un', 'valorUnitario': '2,56', 'valor': '122,88'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Arroz integral', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '10,00', 'unidade': 'Kg', 'valorUnitario': '3,91', 'valor': '39,10'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Paprica picante 094', 'codigoNcmSh': '21039021', 'ncmSh': 'Condimentos e temperos, compostos, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5102', 'quantidade': '1,00', 'unidade': 'Kg', 'valorUnitario': '16,22', 'valor': '16,22'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Oleo de milho 900ml', 'codigoNcmSh': '15079011', 'ncmSh': 'Óleo de soja, refinado, em recipientes com capacidade inferior ou igual a 5 litros', 'cfop': '5102', 'quantidade': '20,00', 'unidade': 'Gfa', 'valorUnitario': '7,71', 'valor': '154,20'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Vinagre balsamico', 'codigoNcmSh': '22090000', 'ncmSh': 'Vinagres e seus sucedâneos obtidos a partir do ácido acético, para usos alimentares', 'cfop': '5405', 'quantidade': '10,00', 'unidade': 'Und', 'valorUnitario': '14,35', 'valor': '143,50'}], 'eventosNotaFiscal': [{'dataEvento': '15/12/2021 15:33:24', 'tipoEvento': 'Ciencia da Operacao', 'evento': 'Protocolo: 891215944725316', 'motivo': ''}]},{'notaFiscalDTO': {'id': 2206466, 'codigoOrgaoSuperiorDestinatario': '52000', 'orgaoSuperiorDestinatario': 'Ministério da Defesa', 'codigoOrgaoDestinatario': '52121', 'orgaoDestinatario': 'Comando do Exército', 'nomeFornecedor': 'SAUDE VIDA COMERCIAL DE ALIMENTOS LTDA - ME', 'cnpjFornecedor': '17.024.148/0001-69', 'municipioFornecedor': 'BRASILIA', 'chaveNotaFiscal': '53220217024148000169550010000021761085002881', 'valorNotaFiscal': '2.569,00', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '15/02/2022 12:44:07', 'dataEmissao': '15/02/2022', 'numero': 2176, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '1', 'descricaoProdutoServico': 'Arroz beneficiado, tipo: japonês', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '6108', 'quantidade': '350,00', 'unidade': 'KG', 'valorUnitario': '7,34', 'valor': '2.569,00'}], 'eventosNotaFiscal': []},{'notaFiscalDTO': {'id': 2303221, 'codigoOrgaoSuperiorDestinatario': '26000', 'orgaoSuperiorDestinatario': 'Ministério da Educação', 'codigoOrgaoDestinatario': '26407', 'orgaoDestinatario': 'Instituto Federal Goiano', 'nomeFornecedor': 'COOPERATIVA AGROPECUARIO DE CAMPOS BELOS E REGIAO', 'cnpjFornecedor': '25.464.746/0001-87', 'municipioFornecedor': 'CAMPOS BELOS', 'chaveNotaFiscal': '52211125464746000187550010000015061369303310', 'valorNotaFiscal': '29.433,70', 'tipoEventoMaisRecente': 'Cancelamento da NF-e', 'dataTipoEventoMaisRecente': '19/11/2021 10:50:57', 'dataEmissao': '19/11/2021', 'numero': 1506, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '10', 'descricaoProdutoServico': 'Sal pct 1kg', 'codigoNcmSh': '111', 'ncmSh': '', 'cfop': '5102', 'quantidade': '119,00', 'unidade': 'UN', 'valorUnitario': '2,36', 'valor': '280,84'}, {'numeroProduto': '15', 'descricaoProdutoServico': 'Extrato de tomate und 340g', 'codigoNcmSh': '106', 'ncmSh': '', 'cfop': '5102', 'quantidade': '240,00', 'unidade': 'UN', 'valorUnitario': '5,71', 'valor': '1.370,40'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'FARINHA DE MANDIOCA PCT 50 GRAMAS', 'codigoNcmSh': '11062000', 'ncmSh': 'Farinhas, sêmolas e pós, de sagu ou das raízes ou tubérculos, da posição 07.14', 'cfop': '5102', 'quantidade': '119,00', 'unidade': 'KG', 'valorUnitario': '4,18', 'valor': '497,42'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'BETERRABA', 'codigoNcmSh': '7069000', 'ncmSh': 'Beterrabas, rabanetes e outras raízes, frescas, refrigeradas', 'cfop': '5102', 'quantidade': '239,00', 'unidade': 'KG', 'valorUnitario': '3,47', 'valor': '829,33'}, {'numeroProduto': '16', 'descricaoProdutoServico': 'ALFACE', 'codigoNcmSh': '7051900', 'ncmSh': 'Outras alfaces frescas ou refrigeradas', 'cfop': '5102', 'quantidade': '479,00', 'unidade': 'UN', 'valorUnitario': '3,09', 'valor': '1.480,11'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Repolho verde', 'codigoNcmSh': '113', 'ncmSh': '', 'cfop': '5102', 'quantidade': '72,00', 'unidade': 'KG', 'valorUnitario': '2,98', 'valor': '214,56'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Arroz Branco pct 5kg', 'codigoNcmSh': '105', 'ncmSh': '', 'cfop': '5102', 'quantidade': '240,00', 'unidade': 'UN', 'valorUnitario': '28,23', 'valor': '6.775,20'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'LARANJA', 'codigoNcmSh': '8051000', 'ncmSh': 'Laranjas, frescas ou secas', 'cfop': '5102', 'quantidade': '478,00', 'unidade': 'KG', 'valorUnitario': '2,46', 'valor': '1.175,88'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'BANANA PRATA', 'codigoNcmSh': '8039000', 'ncmSh': 'Bananas frescas ou secas, exceto bananas-da-terra', 'cfop': '5102', 'quantidade': '239,00', 'unidade': 'UN', 'valorUnitario': '3,55', 'valor': '848,45'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'COUVE', 'codigoNcmSh': '7049090', 'ncmSh': '', 'cfop': '5102', 'quantidade': '239,00', 'unidade': 'UN', 'valorUnitario': '2,49', 'valor': '595,11'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'CENOURA', 'codigoNcmSh': '7061000', 'ncmSh': 'Cenouras e nabos, frescos ou refrigerados', 'cfop': '5102', 'quantidade': '238,00', 'unidade': 'KG', 'valorUnitario': '1,76', 'valor': '418,88'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Feijao Carioca pct 1kg', 'codigoNcmSh': '107', 'ncmSh': '', 'cfop': '5102', 'quantidade': '478,00', 'unidade': 'UN', 'valorUnitario': '8,65', 'valor': '4.134,70'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Macarrao tipo espaguete und 500g', 'codigoNcmSh': '109', 'ncmSh': '', 'cfop': '5102', 'quantidade': '358,00', 'unidade': 'UN', 'valorUnitario': '3,05', 'valor': '1.091,90'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Sardinha em oleo und 125g', 'codigoNcmSh': '112', 'ncmSh': '', 'cfop': '5102', 'quantidade': '1.432,00', 'unidade': 'UN', 'valorUnitario': '5,89', 'valor': '8.434,48'}, {'numeroProduto': '9', 'descricaoProdutoServico': 'Fuba de milho pct 500g', 'codigoNcmSh': '108', 'ncmSh': '', 'cfop': '5102', 'quantidade': '119,00', 'unidade': 'UN', 'valorUnitario': '2,36', 'valor': '280,84'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Oleo de Soja und 900ml', 'codigoNcmSh': '110', 'ncmSh': '', 'cfop': '5102', 'quantidade': '120,00', 'unidade': 'UN', 'valorUnitario': '8,38', 'valor': '1.005,60'}], 'eventosNotaFiscal': [{'dataEvento': '19/11/2021 10:50:57', 'tipoEvento': 'Cancelamento', 'evento': 'Protocolo: 152214586986491', 'motivo': 'VALOR DA NOTA INCORRETO'}]},{'notaFiscalDTO': {'id': 2706614, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36201', 'orgaoDestinatario': 'Fundação Oswaldo Cruz', 'nomeFornecedor': 'WS DISTRIBUIDORA ALIMENTOS LTDA-EPP', 'cnpjFornecedor': '01.220.638/0001-09', 'municipioFornecedor': 'RIO DE JANEIRO', 'chaveNotaFiscal': '33220201220638000109550010000278751000830103', 'valorNotaFiscal': '255,35', 'tipoEventoMaisRecente': 'Manifestação do destinatário - Ciência da operação', 'dataTipoEventoMaisRecente': '16/02/2022 13:31:57', 'dataEmissao': '16/02/2022', 'numero': 27875, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '2', 'descricaoProdutoServico': 'Aveia 250gr', 'codigoNcmSh': '11042200', 'ncmSh': 'Outros grãos trabalhados (por exemplo, descascados, em pérolas, cortados ou partidos), de aveia', 'cfop': '5102', 'quantidade': '1,00', 'unidade': 'Cx', 'valorUnitario': '3,75', 'valor': '3,75'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Arroz Branco', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '65,00', 'unidade': 'Kg', 'valorUnitario': '3,88', 'valor': '252,20'}], 'eventosNotaFiscal': [{'dataEvento': '16/02/2022 13:31:57', 'tipoEvento': 'Ciencia da Operacao', 'evento': 'Protocolo: 891220746893390', 'motivo': ''}]},{'notaFiscalDTO': {'id': 3303769, 'codigoOrgaoSuperiorDestinatario': '36000', 'orgaoSuperiorDestinatario': 'Ministério da Saúde', 'codigoOrgaoDestinatario': '36201', 'orgaoDestinatario': 'Fundação Oswaldo Cruz', 'nomeFornecedor': 'WS DISTRIBUIDORA ALIMENTOS LTDA-EPP', 'cnpjFornecedor': '01.220.638/0001-09', 'municipioFornecedor': 'RIO DE JANEIRO', 'chaveNotaFiscal': '33211101220638000109550010000269691000821040', 'valorNotaFiscal': '2.004,59', 'tipoEventoMaisRecente': 'Manifestação do destinatário - Ciência da operação', 'dataTipoEventoMaisRecente': '24/11/2021 15:33:43', 'dataEmissao': '24/11/2021', 'numero': 26969, 'serie': 1}, 'itensNotaFiscal': [{'numeroProduto': '9', 'descricaoProdutoServico': 'Creme de leite A21', 'codigoNcmSh': '4029900', 'ncmSh': 'Outros leites, cremes de leite, concentrados, adocicados', 'cfop': '5405', 'quantidade': '15,00', 'unidade': 'Und', 'valorUnitario': '5,24', 'valor': '78,60'}, {'numeroProduto': '12', 'descricaoProdutoServico': 'Farinha de rosca', 'codigoNcmSh': '11031100', 'ncmSh': 'Grumos e sêmolas, de trigo', 'cfop': '5102', 'quantidade': '2,00', 'unidade': 'Pct', 'valorUnitario': '4,50', 'valor': '9,00'}, {'numeroProduto': '7', 'descricaoProdutoServico': 'Chocolate granulado', 'codigoNcmSh': '18069000', 'ncmSh': 'Outros chocolates e preparações alimentícias contendo cacau', 'cfop': '5405', 'quantidade': '12,00', 'unidade': 'Un', 'valorUnitario': '3,43', 'valor': '41,16'}, {'numeroProduto': '14', 'descricaoProdutoServico': 'Feijao mulatinho 500gr 024', 'codigoNcmSh': '7133990', 'ncmSh': 'Outros feijões (Vigna ou Phaseolus), secos, em grãos', 'cfop': '5102', 'quantidade': '68,00', 'unidade': 'Pct', 'valorUnitario': '6,69', 'valor': '454,92'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Azeite extra virgem 500ml', 'codigoNcmSh': '15099090', 'ncmSh': 'Outros azeites de oliva', 'cfop': '5405', 'quantidade': '6,00', 'unidade': 'Vd', 'valorUnitario': '13,39', 'valor': '80,34'}, {'numeroProduto': '6', 'descricaoProdutoServico': 'Cha mate tostado  200 gr Prenda', 'codigoNcmSh': '9021000', 'ncmSh': 'Chá verde (não fermentado) em embalagens imediatas de conteúdo não superior a 3 kg', 'cfop': '5102', 'quantidade': '8,00', 'unidade': 'Cx', 'valorUnitario': '7,21', 'valor': '57,68'}, {'numeroProduto': '8', 'descricaoProdutoServico': 'Coco ralado 100gr 126 0', 'codigoNcmSh': '8011100', 'ncmSh': 'Cocos, frescos ou secos, dessecados', 'cfop': '5102', 'quantidade': '6,00', 'unidade': 'Pct', 'valorUnitario': '3,74', 'valor': '22,44'}, {'numeroProduto': '16', 'descricaoProdutoServico': 'ID-57102 Maionese', 'codigoNcmSh': '21039011', 'ncmSh': 'Maionese, em embalagens imediatas de conteúdo inferior ou igual a 1 kg', 'cfop': '5102', 'quantidade': '5,00', 'unidade': 'Und', 'valorUnitario': '6,61', 'valor': '33,05'}, {'numeroProduto': '10', 'descricaoProdutoServico': 'Ervilha em conserva', 'codigoNcmSh': '7102100', 'ncmSh': 'Ervilhas, não cozidas ou cozidas em água ou vapor, congeladas', 'cfop': '5405', 'quantidade': '6,00', 'unidade': 'Lta', 'valorUnitario': '2,25', 'valor': '13,50'}, {'numeroProduto': '18', 'descricaoProdutoServico': 'Polvilho Azedo 500 GR', 'codigoNcmSh': '11081400', 'ncmSh': 'Fécula de mandioca', 'cfop': '5102', 'quantidade': '5,00', 'unidade': 'Pct', 'valorUnitario': '7,26', 'valor': '36,30'}, {'numeroProduto': '17', 'descricaoProdutoServico': 'Molho', 'codigoNcmSh': '20029090', 'ncmSh': 'Outros tomates preparados ou conservados, exceto em vinagre ou em ácido acético', 'cfop': '5102', 'quantidade': '7,00', 'unidade': 'Un', 'valorUnitario': '4,63', 'valor': '32,41'}, {'numeroProduto': '15', 'descricaoProdutoServico': 'Feijao preto', 'codigoNcmSh': '7133319', 'ncmSh': 'Outros feijões comuns, pretos, secos, em grãos', 'cfop': '5102', 'quantidade': '50,00', 'unidade': 'Kg', 'valorUnitario': '7,22', 'valor': '361,00'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Amido de milho', 'codigoNcmSh': '11081200', 'ncmSh': 'Amido de milho', 'cfop': '5102', 'quantidade': '1,00', 'unidade': 'Un', 'valorUnitario': '8,49', 'valor': '8,49'}, {'numeroProduto': '11', 'descricaoProdutoServico': 'Aveia 250gr', 'codigoNcmSh': '11042200', 'ncmSh': 'Outros grãos trabalhados (por exemplo, descascados, em pérolas, cortados ou partidos), de aveia', 'cfop': '5102', 'quantidade': '4,00', 'unidade': 'Cx', 'valorUnitario': '3,75', 'valor': '15,00'}, {'numeroProduto': '5', 'descricaoProdutoServico': 'Canela em po 50gr 047', 'codigoNcmSh': '9062000', 'ncmSh': 'Canela e flores de caneleira, trituradas ou em pó', 'cfop': '5102', 'quantidade': '1,00', 'unidade': 'Un', 'valorUnitario': '4,65', 'valor': '4,65'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Arroz Branco', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '120,00', 'unidade': 'Kg', 'valorUnitario': '3,88', 'valor': '465,60'}, {'numeroProduto': '19', 'descricaoProdutoServico': 'Requeijao 021', 'codigoNcmSh': '4052000', 'ncmSh': 'Pasta de espalhar (pasta de barrar) de produtos provenientes do leite', 'cfop': '5102', 'quantidade': '5,00', 'unidade': 'Cpo', 'valorUnitario': '5,98', 'valor': '29,90'}, {'numeroProduto': '4', 'descricaoProdutoServico': 'Chocolate em po 200gr 004', 'codigoNcmSh': '18063210', 'ncmSh': 'Chocolate não recheado, em tabletes, barras e paus', 'cfop': '5405', 'quantidade': '3,00', 'unidade': 'Cx', 'valorUnitario': '24,55', 'valor': '73,65'}, {'numeroProduto': '20', 'descricaoProdutoServico': 'Gelatina de uva 004', 'codigoNcmSh': '21069029', 'ncmSh': 'Pós para preparações de cremes, sorvetes, flans, gelatinas ou preparações similares', 'cfop': '5405', 'quantidade': '9,00', 'unidade': 'Kg', 'valorUnitario': '10,60', 'valor': '95,40'}, {'numeroProduto': '13', 'descricaoProdutoServico': 'Farinha de trigo', 'codigoNcmSh': '11010010', 'ncmSh': 'Farinha de trigo', 'cfop': '5405', 'quantidade': '25,00', 'unidade': 'Kg', 'valorUnitario': '3,66', 'valor': '91,50'}], 'eventosNotaFiscal': [{'dataEvento': '24/11/2021 15:33:43', 'tipoEvento': 'Ciencia da Operacao', 'evento': 'Protocolo: 891215526311009', 'motivo': ''}]},{'notaFiscalDTO': {'id': 3703421, 'codigoOrgaoSuperiorDestinatario': '-1', 'orgaoSuperiorDestinatario': 'Sem informação', 'codigoOrgaoDestinatario': '-1', 'orgaoDestinatario': 'Sem informação', 'nomeFornecedor': 'COOP PROD RURAIS AGRIC FAMILIAR DE FW', 'cnpjFornecedor': '10.378.011/0001-63', 'municipioFornecedor': 'FREDERICO WESTPHALEN', 'chaveNotaFiscal': '43211110378011000163550000000021521927364159', 'valorNotaFiscal': '14.161,16', 'tipoEventoMaisRecente': 'Autorização de Uso', 'dataTipoEventoMaisRecente': '22/11/2021 14:17:10', 'dataEmissao': '22/11/2021', 'numero': 2152, 'serie': 0}, 'itensNotaFiscal': [{'numeroProduto': '4', 'descricaoProdutoServico': 'Mamao', 'codigoNcmSh': '8072000', 'ncmSh': 'Mamões (papaias) frescos', 'cfop': '5102', 'quantidade': '452,00', 'unidade': 'KG', 'valorUnitario': '5,77', 'valor': '2.608,04'}, {'numeroProduto': '2', 'descricaoProdutoServico': 'Banana Prata', 'codigoNcmSh': '8031000', 'ncmSh': 'Bananas-da-terra, frescas ou secas', 'cfop': '5102', 'quantidade': '452,00', 'unidade': 'KG', 'valorUnitario': '5,23', 'valor': '2.363,96'}, {'numeroProduto': '1', 'descricaoProdutoServico': 'Farinha de Milho Di Bernardo', 'codigoNcmSh': '11022000', 'ncmSh': 'Farinha de milho', 'cfop': '5102', 'quantidade': '678,00', 'unidade': 'KG', 'valorUnitario': '4,66', 'valor': '3.159,48'}, {'numeroProduto': '3', 'descricaoProdutoServico': 'Arroz Branco 2k', 'codigoNcmSh': '10063021', 'ncmSh': 'Arroz semibranqueado ou branqueado, não parboilizado, polido ou brunido', 'cfop': '5102', 'quantidade': '904,00', 'unidade': 'UN', 'valorUnitario': '6,67', 'valor': '6.029,68'}], 'eventosNotaFiscal': []},], Responda o seguinte questionamento: qual e o valor unitario desses produtos?

------


> **model:** - Azeite De Dende 100ml Cepera UN: 6,65
> - Molho De Tomate 340g Tradicional Quero UN: 1,79
> - Creme De Leite 200g Leve Tp Terra Viva UN: 2,27
> - Canela Em Po 30gr Tubo Kamila UN: 2,55
> - Cenoura Kg Feira Quadri KG: 6,65
> - Ervilha 300g Pct / Congelado Lar* UN: 7,60
> - Feijão Kg Feira Quadri KG: 4,52
> - Leite 1 Lt Zero Lactose Terra Viva UN: 3,12
> - Chocolate Granulado 50g Kamila UN: 1,89
> - Peito De Frango Kg C/ Osso Quadri. UN: 12,34
> - Vagem Kg Feira Quadri KG: 13,33
> - Tomate Kg Longa Vida Feira Quadri KG: 1,93
> - Maionese 500g Pet/ Light Hellmanns UN: 8,54
> - Iogurte Copo 165g Natural Integral Frimesa UN: 2,27
> - Alface Un Crespa Feira Quadri UN: 3,79
> - Farinha De Trigo 5kg Bonina UN: 16,61
> - Farinha De Trigo 1kg Integral Anaconda UN: 3,79
> - Farinha De Milho 500g Mimoso Zaeli UN: 2,74
> - Sabao em Pedra com 05 KG: 4,20
> - Crepe de aco com 08 KG: 92,00
> - Esponja de aco com 08 KG: 3,31
> - Detergente 500ML KG: 1,78
> - Pasta de dente 90G KG: 2,52
> - Po P/ Gelatina 24g Sem Sabor Bretzke UN: 4,74
> - Manjericao 15g Kamila UN: 1,89
> - Sabao em Po 1KG KG: 889,60
> - Composto Lacteo 200G KG: 889,60
> - Leite Condensado 395G KG: 3,78
> - Vinagre 750ML KG: 1,00
> - Biscoito Salgado 200G KG: 1,98
> - Goiabada 300G KG: 2,76
> - Achocolatado 200G KG: 3,23
> - Farofa Mandioca 250G KG: 2,20
> - Aveia 200G UN: 3,85
> - Esponja Multiuso KG: 1,78
> - Embalagem Plastica com Alça Amarela LT: 0,50
> - Oleo de Soja GFA 900 ML LT: 10,45
> - Arroz Tipo 1 5KG KG: 22,00
> - Cafe 500G KG: 5,55
> - Acucar 1KG KG: 3,54
> - Farinha Branca Seca UN: 6,50
> - Polpa de Goiaba KG: 10,30
> - Polpa de Caju KG: 10,00
> - Raiz de Mandioca (macaxeira) KG: 4,15
> - Mamao Papaya KG: 3,42
> - Abobora KG: 4,05
> - Polpa de Acerola KG: 10,00
> - Polpa de caja KG: 15,16
> - Banana KG: 4,15
> - Feijao KG: 10,00
> - Arroz KG: 5,65
> - Chocolate em po 200gr Cx: 24,55
> - Aveia 250gr Cx: 3,75
> - Arroz Branco Kg: 4,18
> - copo descartavel PT: 4,00
> - Biscoito salgado integral UN: 9,80
> - pano de chao UN: 4,98
> - Pimenta do reino preta ptc 50g PT: 3,44
> - Alho in natura KG: 15,48
> - Macarrao de Arroz KG: 6,50
> - Linguica tipo Calabresa KG: 14,50
> - polpa de fruta acerola 1kg KG: 14,18
> - Massa seca p Lasanha KG: 10,99
> - Batata Doce in Natura KG: 4,95
> - Beterraba in Natura KG: 5,30
> - Tangerina KG: 5,80
> - oregano pct 250g PT: 7,73
> - Abacaxi UN: 5,00
> - Maca KG: 5,80
> - Mamao KG: 6,00
> - Banana Prata KG: 4,77
> - Arroz Branco 2k UN: 10,00
> - Pao Caseiro Und: 6,36
> - Banana Prata KG: 3,96
> - Acucara Mascavo Piaia KG: 10,08
> - Queijo Colonial Eduvavi KG: 33,93
> - Milho Pipoca KG: 3,79
> - Salame Bisolo KG: 14,99
> - Bacon Defumado UN: 14,36
> - Carne Suina Costeliha Defumada UN: 14,372
> - Suco de Uva Integral Vitis UN: 17,66
> - Feijao Preto KG: 8,93
> - Arroz Branco 5kg UN: 27,09
> - Laranja Tipo Suco KG: 2,88
> - Canjica Amarela Progresso 500g PCTE: 3,58
> - Bolacha de Polvilho KG: 5,83
> - Mel Rainha Alimentos 500 G UN: 13,66
> - Farinha de Milho Di Bernardo KG: 4,49
> - Ovos Tradicao Caipira DUZIA: 8,26
> - Bolacha Caseira de Acucar Mascavo KG: 5,83
> - Brocolis UN: 5,50
> - Batata Doce KG: 3,98
> - Creme de leite A21 Und: 5,24
> - Feijao branco 500gr Pct: 6,51
> - Feijao preto KG: 7,22
> - Arroz Branco Kg: 3,88
> - Queijo creme cheese 174 Un: 9,90
> - Catchup 200gr Un: 2,68
> - Mostarda Un: 3,21
> - Molho de tomate 340gr Un: 2,41
> - Gelatina de limao a21 . Un: 1,28
> - ID-59598 Extrato de tomate Un: 2,56
> - Arroz integral Kg: 3,91
> - Paprica picante 094 KG: 16,22
> - Oleo de milho 900ml Gfa: 7,71
> - Vinagre balsamico Und: 14,35
> - Arroz beneficiado, tipo: japonês KG: 7,34
> - Sal pct 1kg UN: 2,36
> - Extrato de tomate und 340g UN: 5,71
> - FARINHA DE MANDIOCA PCT 50 GRAMAS KG: 4,18
> - BETERRABA KG: 3,47
> - ALFACE UN: 3,09
> - Repolho verde KG: 2,98
> - Arroz Branco pct 5kg UN: 28,23
> - LARANJA KG: 2,46
> - BANANA PRATA UN: 3,55
> - COUVE UN: 2,49
> - CENOURA KG: 1,76
> - Feijao Carioca pct 1kg UN: 8,65
> - Macarrao tipo espaguete und 500g UN: 3,05
> - Sardinha em oleo und 125g UN: 5,89
> - Fuba de milho pct 500g UN: 2,36
> - Oleo de Soja und 900ml UN: 8,38
> - Aveia 250gr Cx: 3,75
> - Arroz Branco Kg: 3,88
> - Creme de leite A21 Und: 5,24

------
